In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tabulate import tabulate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_log_error, mean_absolute_error, make_scorer
from sklearn.model_selection import train_test_split

data = pd.read_csv('csv/final_dataset.csv')
print(data.columns)

# Separate features and response variables
X = data.iloc[:, 2:]                                # features
Y = data['temp_measured']                           # response variable: geothermal reservoir measured temperature
print(f'Features of dataset: {X.columns}')
print(f'Number of compenents in features: {X.shape[1]}')
print(Y.head(10))

Index(['well_sample', 'temp_measured', 'pH', 'Na ', 'K', 'Ca', 'Mg', 'Cl',
       'SO4'],
      dtype='object')
Features of dataset: Index(['pH', 'Na ', 'K', 'Ca', 'Mg', 'Cl', 'SO4'], dtype='object')
Number of compenents in features: 7
0    137
1    137
2    137
3    137
4    150
5    116
6    165
7    140
8    115
9    115
Name: temp_measured, dtype: int64


In [ ]:
### Desicion tree with GridSearchCV for parameters tuning

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

start_time_dt_gs = time.time()

x_train_dt, x_test_dt, y_train_dt, y_test_dt = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train_dt = scaler.fit_transform(x_train_dt)
x_test_dt = scaler.transform(x_test_dt)

# Setting up grid search cross validation for parameters tuning
# param_grid_dt: dictionary of parameters to be tested.
# cv: number of split for cross-validation: 5- fold cross-validation (validación cruzada quíntuple).
# scoring or evaluation metrics: mean squeared error. GridSearchCV maximize the scoring metrics, that's why it's called 'neg_mean_squared_error'.
# The actual calculation is: -1 * mean_squared_error

# Parameters tuning
tree_regressor = DecisionTreeRegressor()
param_grid_dt = {'criterion': ['squared_error'],
            'max_depth': [50, 75, 100],
            'min_samples_split': [5, 10, 25],
            'max_leaf_nodes': [25, 50, 75],
            'min_impurity_decrease': [0, 0.01, 0.1]}

grid_search_cv = GridSearchCV(
    estimator=tree_regressor, 
    param_grid=param_grid_dt, 
    cv=5, 
    verbose=1, 
    scoring='neg_mean_squared_error',
    n_jobs=-1                                         # n_jobs=-1 utilize all the cores avalaible,
    )


grid_search_cv.fit(x_train_dt, y_train_dt)
best_model_dt = grid_search_cv.best_estimator_

print('='*75)
print(f'Desicion tree regressor best parameters: \n{best_model_dt}')
print('='*75)

y_pred_test_dt = best_model_dt.predict(x_test_dt)
y_pred_train_dt = best_model_dt.predict(x_train_dt)

end_time_dt_gs = time.time()
training_time_dt_gs = end_time_dt_gs - start_time_dt_gs

def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_dt = r2_score(y_test_dt, y_pred_test_dt)
mse_dt = mean_squared_error(y_test_dt, y_pred_test_dt)
mslr_dt = mean_squared_log_error(y_test_dt, y_pred_test_dt)
mae_dt = mean_absolute_error(y_test_dt, y_pred_test_dt)
mrse_dt = mean_relative_squared_error(y_test_dt, y_pred_test_dt)

eval_metrics_dt = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'Desicion_tree_GS': [r2_dt, mse_dt, mae_dt, mslr_dt, mrse_dt, training_time_dt_gs]
}

df_metrics_dt = pd.DataFrame(eval_metrics_dt)
# df_metrics_dt.to_csv('metrics_dt.csv', index=False)

print(tabulate(df_metrics_dt.round(4), headers='keys', tablefmt='pretty', showindex=False))

#joblib.dump(tree_regressor, 'dt_model.joblib')
#print("Decision Tree model saved as 'decision_tree_model.joblib'.")

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Desicion tree regressor best parameters: 
DecisionTreeRegressor(max_depth=100, max_leaf_nodes=75,
                      min_impurity_decrease=0.1, min_samples_split=5)
+---------------+------------------+
| Eval_metrics  | Desicion_tree_GS |
+---------------+------------------+
|   R2 Score    |      0.7965      |
|      MSE      |    1535.4578     |
|      MAE      |      21.46       |
|     MSLE      |      0.1612      |
|     MRSE      |      1.3229      |
| Training time |      0.2217      |
+---------------+------------------+


In [23]:
### Decision tree with RandomizedSearchCV for parameters tuning

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

start_time_dt_rs = time.time()

x_train_dt, x_test_dt, y_train_dt, y_test_dt = train_test_split(X, Y, test_size=0.20, random_state=42)

scaler = StandardScaler()
x_train_dt = scaler.fit_transform(x_train_dt)
x_test_dt = scaler.transform(x_test_dt)

# Parameters tuning with distributions for RandomizedSearchCV
tree_regressor = DecisionTreeRegressor()
param_distributions_dt = {
    'criterion': ['squared_error'],
    'max_depth': randint(50, 100),             
    'min_samples_split': randint(5, 25),        
    'max_leaf_nodes': randint(25, 75),          
    'min_impurity_decrease': uniform(0, 0.1)    
}

random_search_cv = RandomizedSearchCV(
    estimator=tree_regressor, 
    param_distributions=param_distributions_dt, 
    n_iter=50,  # Number of parameter settings sampled
    cv=5, 
    verbose=1, 
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

random_search_cv.fit(x_train_dt, y_train_dt)
best_model_dt = random_search_cv.best_estimator_
print('='*75)
print(f'Decision tree regressor best parameters: \n{best_model_dt}')
print('='*75)
y_pred_test_dt = best_model_dt.predict(x_test_dt)
y_pred_train_dt = best_model_dt.predict(x_train_dt)

end_time_dt_rs = time.time()
training_time_dt_rs = end_time_dt_rs - start_time_dt_rs

def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_dt = r2_score(y_test_dt, y_pred_test_dt)
mse_dt = mean_squared_error(y_test_dt, y_pred_test_dt)
mslr_dt = mean_squared_log_error(y_test_dt, y_pred_test_dt)
mae_dt = mean_absolute_error(y_test_dt, y_pred_test_dt)
mrse_dt = mean_relative_squared_error(y_test_dt, y_pred_test_dt)

eval_metrics_dt = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'Decision_tree_RS': [r2_dt, mse_dt, mae_dt, mslr_dt, mrse_dt, training_time_dt_rs]
}

df_metrics_dt = pd.DataFrame(eval_metrics_dt)
# df_metrics_dt.to_csv('metrics_dt_randomized.csv', index=False)

print(tabulate(df_metrics_dt.round(4), headers='keys', tablefmt='pretty', showindex=False))

#joblib.dump(best_model_dt, 'dt_randomized_model.joblib')
#print("Decision Tree model saved as 'dt_randomized_model.joblib'.")

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Decision tree regressor best parameters: 
DecisionTreeRegressor(max_depth=84, max_leaf_nodes=68,
                      min_impurity_decrease=0.06375574713552132,
                      min_samples_split=7)
+---------------+------------------+
| Eval_metrics  | Decision_tree_RS |
+---------------+------------------+
|   R2 Score    |      0.8182      |
|      MSE      |    1371.8661     |
|      MAE      |     18.7549      |
|     MSLE      |      0.1473      |
|     MRSE      |      1.2954      |
| Training time |      0.1623      |
+---------------+------------------+


In [ ]:
### Decision Tree with Optuna hyperparameter optimization

import optuna
from sklearn.model_selection import cross_val_score

start_time_dt_op = time.time()

x_train_dt, x_test_dt, y_train_dt, y_test_dt = train_test_split(X, Y, test_size=0.20, random_state=42)

scaler = StandardScaler()
x_train_dt = scaler.fit_transform(x_train_dt)
x_test_dt = scaler.transform(x_test_dt)

def objective_dt(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 50, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 5, 25),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 25, 75),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.1)
    }
    model = DecisionTreeRegressor(**params, random_state=42)
    score = cross_val_score(model, x_train_dt, y_train_dt, cv=5, scoring='neg_mean_squared_error').mean()
    return -score

optuna_dt = optuna.create_study(direction='minimize')
optuna_dt.optimize(objective_dt, n_trials=40, show_progress_bar=False)

print('='*125)
print(f"Best Decision Tree params: \n{optuna_dt.best_params}")
print('='*125)

# Train and evaluate the best model
best_dt = DecisionTreeRegressor(**optuna_dt.best_params, random_state=42)
best_dt.fit(x_train_dt, y_train_dt)
y_pred_dt = best_dt.predict(x_test_dt)

final_time_dt_op = time.time()
training_time_dt_op = final_time_dt_op - start_time_dt_op



# Metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_squared_log_error

def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_dt = r2_score(y_test_dt, y_pred_dt)
mse_dt = mean_squared_error(y_test_dt, y_pred_dt)
mae_dt = mean_absolute_error(y_test_dt, y_pred_dt)
msle_dt = mean_squared_log_error(y_test_dt, y_pred_dt)
mrse_dt = mean_relative_squared_error(y_test_dt, y_pred_dt)

dt_eval_metrics = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'Decision_tree_Op': [r2_dt, mse_dt, mae_dt, msle_dt, mrse_dt, training_time_dt_op]
}

dt_df_metrics = pd.DataFrame(dt_eval_metrics)
# dt_df_metrics.to_csv('dt_optuna_metrics.csv', index=False)

print(tabulate(dt_df_metrics.round(4), headers='keys', tablefmt='pretty', showindex=False))

[I 2025-08-25 08:59:57,062] A new study created in memory with name: no-name-39d02e8e-afc7-4a64-9bc7-8242f217b1ea
[I 2025-08-25 08:59:57,076] Trial 0 finished with value: 2299.4480735074253 and parameters: {'max_depth': 69, 'min_samples_split': 23, 'max_leaf_nodes': 27, 'min_impurity_decrease': 0.06764553656256393}. Best is trial 0 with value: 2299.4480735074253.
[I 2025-08-25 08:59:57,089] Trial 1 finished with value: 2041.0919765276747 and parameters: {'max_depth': 59, 'min_samples_split': 11, 'max_leaf_nodes': 53, 'min_impurity_decrease': 0.09423536660249288}. Best is trial 1 with value: 2041.0919765276747.
[I 2025-08-25 08:59:57,111] Trial 2 finished with value: 2309.653616644655 and parameters: {'max_depth': 69, 'min_samples_split': 25, 'max_leaf_nodes': 51, 'min_impurity_decrease': 0.01511041304352937}. Best is trial 1 with value: 2041.0919765276747.
[I 2025-08-25 08:59:57,131] Trial 3 finished with value: 2293.336545971303 and parameters: {'max_depth': 81, 'min_samples_split': 2

Best Decision Tree params: 
{'max_depth': 84, 'min_samples_split': 6, 'max_leaf_nodes': 25, 'min_impurity_decrease': 0.02447609527001826}
+---------------+------------------+
| Eval_metrics  | Decision_tree_Op |
+---------------+------------------+
|   R2 Score    |      0.7276      |
|      MSE      |    2055.0446     |
|      MAE      |     23.7536      |
|     MSLE      |      0.1663      |
|     MRSE      |      1.3511      |
| Training time |      0.9278      |
+---------------+------------------+


In [37]:
### Random Forest with RandomizedSearchCV for hyperparameter optimization

from sklearn.ensemble import RandomForestRegressor
import time

start_time_rf_rs = time.time()

x_train_rf, x_test_rf, y_train_rf, y_test_rf = train_test_split(X, Y, test_size=0.20, random_state=42)

scaler = StandardScaler()
x_train_rf = scaler.fit_transform(x_train_rf)
x_test_rf = scaler.transform(x_test_rf)

# Define parameter distributions for RandomizedSearchCV
rf_param_distributions = {
    'n_estimators': randint(50, 500),
    'max_depth': randint(5, 50),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

# RandomizedSearchCV with Random Forest
rf_regressor = RandomForestRegressor(random_state=42, n_jobs=-1)

random_search_rf = RandomizedSearchCV(
    estimator=rf_regressor,
    param_distributions=rf_param_distributions,
    n_iter=50,  # Number of parameter settings sampled
    cv=5,
    verbose=1,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

random_search_rf.fit(x_train_rf, y_train_rf)
best_rf = random_search_rf.best_estimator_

print("="*135)
print(f"Best Random Forest params: \n{random_search_rf.best_params_}")
print("="*135)
y_pred_rf = best_rf.predict(x_test_rf)

end_time_rf_rs = time.time()
training_time_rf_rs= end_time_rf_rs - start_time_rf_rs

def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_rf = r2_score(y_test_rf, y_pred_rf)
mse_rf = mean_squared_error(y_test_rf, y_pred_rf)
mae_rf = mean_absolute_error(y_test_rf, y_pred_rf)
msle_rf = mean_squared_log_error(y_test_rf, y_pred_rf)
mrse_rf = mean_relative_squared_error(y_test_rf, y_pred_rf)

rf_eval_metrics = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'Random_forest_RS': [r2_rf, mse_rf, mae_rf, msle_rf, mrse_rf, training_time_rf_rs]
}

rf_df_metrics = pd.DataFrame(rf_eval_metrics)
#rf_df_metrics.to_csv('rf_randomized_metrics.csv', index=False)

print(tabulate(rf_df_metrics.round(4), headers='keys', tablefmt='pretty', showindex=False))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Random Forest params: 
{'bootstrap': False, 'max_depth': 34, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 363}
+---------------+------------------+
| Eval_metrics  | Random_forest_RS |
+---------------+------------------+
|   R2 Score    |      0.7983      |
|      MSE      |    1521.8001     |
|      MAE      |      21.442      |
|     MSLE      |      0.1257      |
|     MRSE      |      0.7566      |
| Training time |     11.8294      |
+---------------+------------------+


In [38]:
### Random Forest with Optuna hyperparameter optimization

start_time_rf_op = time.time()

x_train_rf, x_test_rf, y_train_rf, y_test_rf = train_test_split(X, Y, test_size=0.20, random_state=42)

scaler = StandardScaler()
x_train_rf = scaler.fit_transform(x_train_rf)
x_test_rf = scaler.transform(x_test_rf)

def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }
    model = RandomForestRegressor(**params, random_state=42, n_jobs=-1)
    score = cross_val_score(model, x_train_rf, y_train_rf, cv=5, scoring='neg_mean_squared_error').mean()
    return -score

study_rf = optuna.create_study(direction='minimize')
study_rf.optimize(objective_rf, n_trials=40)

print("Best Random Forest params:", study_rf.best_params)

# Train and evaluate the best model
best_rf = RandomForestRegressor(**study_rf.best_params, random_state=42, n_jobs=-1)
best_rf.fit(x_train_rf, y_train_rf)
y_pred_rf = best_rf.predict(x_test_rf)

end_time_rf_op = time.time()
training_time_rf_op = end_time_rf_op - start_time_rf_op

def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_rf = r2_score(y_test_rf, y_pred_rf)
mse_rf = mean_squared_error(y_test_rf, y_pred_rf)
mae_rf = mean_absolute_error(y_test_rf, y_pred_rf)
msle_rf = mean_squared_log_error(y_test_rf, y_pred_rf)
mrse_rf = mean_relative_squared_error(y_test_rf, y_pred_rf)

rf_eval_metrics = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'Random_forest_Op': [r2_rf, mse_rf, mae_rf, msle_rf, mrse_rf, training_time_rf_op]
}

rf_df_metrics = pd.DataFrame(rf_eval_metrics)
rf_df_metrics.to_csv('rf_metrics.csv', index=False)

print(tabulate(rf_df_metrics.round(4), headers='keys', tablefmt='pretty', showindex=False))

[I 2025-08-25 09:18:46,005] A new study created in memory with name: no-name-4b883009-6814-4880-9928-28baff4e0ac7
[I 2025-08-25 09:18:47,265] Trial 0 finished with value: 1468.2976793751936 and parameters: {'n_estimators': 219, 'max_depth': 32, 'min_samples_split': 17, 'min_samples_leaf': 6, 'max_features': 'log2', 'bootstrap': True}. Best is trial 0 with value: 1468.2976793751936.
[I 2025-08-25 09:18:48,044] Trial 1 finished with value: 1636.548003231111 and parameters: {'n_estimators': 121, 'max_depth': 30, 'min_samples_split': 11, 'min_samples_leaf': 8, 'max_features': 'log2', 'bootstrap': True}. Best is trial 0 with value: 1468.2976793751936.
[I 2025-08-25 09:18:50,259] Trial 2 finished with value: 1073.6154900206986 and parameters: {'n_estimators': 412, 'max_depth': 9, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 'log2', 'bootstrap': True}. Best is trial 2 with value: 1073.6154900206986.
[I 2025-08-25 09:18:51,322] Trial 3 finished with value: 2355.8222211250363 

Best Random Forest params: {'n_estimators': 457, 'max_depth': 50, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': False}
+---------------+------------------+
| Eval_metrics  | Random_forest_Op |
+---------------+------------------+
|   R2 Score    |      0.8106      |
|      MSE      |    1428.6529     |
|      MAE      |     19.9853      |
|     MSLE      |      0.1257      |
|     MRSE      |      0.7896      |
| Training time |      56.424      |
+---------------+------------------+


In [39]:
### XGBoost implementation with RandomizedSearchCV for faster hyperparametr tuning
'''
Unlike GridSearchCV, which tries all possible combiationn of hyperparameters,
RandomizedSearchCV samples a fixed number of hyperparameters combinations from
the specified space. This is useful when you have a large hyprparameteres space,
as it saves time by exploring only a subset of all possible combinations.
'''

import xgboost as xgb
import json

start_time_xg_rs = time.time()

x_train_xg, x_test_xg, y_train_log_xg, y_test_log_xg = train_test_split(X, np.log(Y), test_size = 0.20, random_state = 45)

scaler = StandardScaler()
x_train_xg = scaler.fit_transform(x_train_xg)
x_test_xg = scaler.transform(x_test_xg)

# Define the XGBoost regressor

xgb_regressor = xgb.XGBRegressor() # (tree_method='gpu_hist', predictor='gpu_predictor', use_label_encoder=False)   # Use GPU for training

# Conveert data into DMatrix (XGBoost optimized data structure)
dtrain = xgb.DMatrix(x_train_xg, label=y_train_log_xg)
dtest = xgb.DMatrix(x_test_xg, label=y_test_log_xg)

# Define the space of hyperparameters
xgb_param_distributions = {
    'colsample_bytree': uniform(0.3, 0.7),
    'learning_rate': uniform(0.00001, 1),
    'max_depth': randint(10, 100),
    'n_estimators': randint(50, 5000),
    'subsample': uniform(0.1, 0.7),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1)
}

# Define scoring functions
scorers = {
    'r2': make_scorer(r2_score),
    'neg_mse': make_scorer(mean_squared_error, greater_is_better=False)
}

# RandomizedSearchCV with cross-validation to optimize hyperparameter tuning
random_search_xgb = RandomizedSearchCV(
    estimator=xgb_regressor,
    param_distributions=xgb_param_distributions,
    scoring=scorers,
    refit='r2',         # Refit the model using R² after searching
    cv=10,              # 10-fold cross-validation
    n_iter=100,          # Number of random combinations to try
    verbose=1,
    random_state=42,
    n_jobs=-1           # -1 to use all processors avalaible
)

# Fit the RSCV to find the best parameters
random_search_xgb.fit(x_train_xg, y_train_log_xg)

# Get the best parameters
xgb_best_params = random_search_xgb.best_params_

print("="*200)
print(f'Best XGBoost hyperparameters: \n{xgb_best_params}')
print("="*200)

params_filename = 'xgbRS_best_params.json'
with open(params_filename, 'w') as params_file:
    json.dump(xgb_best_params, params_file)
print(f'Best XGBoost hyperparameters saved to: \n{params_filename}')


# Instantiate and train the XGBoost regressor with the best hyperparameters 
xgb_regressor = xgb.XGBRegressor(**xgb_best_params, 
                                 eval_metrics='rmse', 
                                 early_stopping_rounds=1500) #, tree_method='gpu_hist', predictor='gpu_predictor',)  ## Use GPU for training

xgb_model = xgb_regressor.fit(x_train_xg, 
                              y_train_log_xg, 
                              verbose=False, 
                              eval_set = [(x_train_xg, y_train_log_xg), (x_test_xg, y_test_log_xg)])

end_time_xg_rs = time.time()
training_time_xgb_rs = end_time_xg_rs - start_time_xg_rs

# Save the trained XGBoost model to a file
# model_filename = 'xgbRS_model.json'
# xgb_model.save_model(model_filename)
# print(f'XGBoost model saved to {model_filename}')

# Predict on test data
y_pred_log_test_xg = xgb_model.predict(x_test_xg)
y_pred_log_train_xg = xgb_model.predict(x_train_xg)

# Return predictions to the original scale
y_pred_test_xg = np.exp(y_pred_log_test_xg)
y_pred_train_xg = np.exp(y_pred_log_train_xg)
y_train_xg = np.exp(y_train_log_xg)
y_test_xg = np.exp(y_test_log_xg)

# Evaluate the model
# Define custom evaluation function: Mean Relative Squared Error (MRSE)
def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_xgb= r2_score(y_test_xg, y_pred_test_xg)
mse_xgb = mean_squared_error(y_test_xg, y_pred_test_xg)
mae_xgb = mean_absolute_error(y_test_xg, y_pred_test_xg)
mslr_xgb = mean_squared_log_error(y_test_xg, y_pred_test_xg)
mrse_xgb = mean_relative_squared_error(y_test_xg, y_pred_test_xg)


xgb_eval_metrics = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'XGBoost_RS': [r2_xgb, mse_xgb, mae_xgb, mslr_xgb, mrse_xgb, training_time_xgb_rs]
}

xgb_df_metrics = pd.DataFrame(xgb_eval_metrics)
# xgb_df_metrics.to_csv('xgb_metrics.csv', index=False)

print(tabulate(xgb_df_metrics.round(4), headers='keys', tablefmt='pretty', showindex=False))

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Best XGBoost hyperparameters: 
{'colsample_bytree': 0.7395302726436355, 'learning_rate': 0.0817690319488719, 'max_depth': 72, 'n_estimators': 4193, 'reg_alpha': 0.06107795985486375, 'reg_lambda': 0.2768776481472037, 'subsample': 0.6643408958551429}
Best XGBoost hyperparameters saved to: 
xgbRS_best_params.json


/home/chris/venv/cdd/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning:

[09:20:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "eval_metrics" } are not used.




+---------------+------------+
| Eval_metrics  | XGBoost_RS |
+---------------+------------+
|   R2 Score    |   0.9387   |
|      MSE      |  443.9573  |
|      MAE      |  14.2474   |
|     MSLE      |   0.0278   |
|     MRSE      |   0.0389   |
| Training time |  45.5416   |
+---------------+------------+


In [40]:
### XGBoost with Optuna hyperparameter optimization

start_time_xg_op = time.time()

x_train_xg, x_test_xg, y_train_log_xg, y_test_log_xg = train_test_split(X, np.log(Y), test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train_xg = scaler.fit_transform(x_train_xg)
x_test_xg = scaler.transform(x_test_xg)

def objective_xgb(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 5, 100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.1, 0.8),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True)
    }


    model = xgb.XGBRegressor(**params, random_state=42)
    score = cross_val_score(model, x_train_xg, y_train_log_xg, 
                          cv=5, scoring='neg_mean_squared_error').mean()
    return -score

# Create and optimize study
study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(objective_xgb, n_trials=50)

print("="*100)
print(f"Best XGBoost parameters: \n{study_xgb.best_params}")
print("="*100)

params_filename = 'xgb_optuna_best_params.json'
with open(params_filename, 'w') as params_file:
    json.dump(study_xgb.best_params, params_file)
print(f'Best parameters saved to: {params_filename}')

# Train model with best parameters
best_xgb = xgb.XGBRegressor(**study_xgb.best_params, 
                            random_state=42,  
                            eval_metrics='rmse', 
                            early_stopping_rounds=1500) #, tree_method='gpu_hist', predictor='gpu_predictor',)  ## Use GPU for training)

best_xgb.fit(x_train_xg, 
             y_train_log_xg,
             verbose = True,
             eval_set = [(x_train_xg, y_train_log_xg), (x_test_xg, y_test_log_xg)]
             )
'''
# Instantiate and train the XGBoost regressor with the best hyperparameters 
xgb_regressor = xgb.XGBRegressor(**xgb_best_params, 
                                 eval_metrics='rmse', 
                                 early_stopping_rounds=1500) #, tree_method='gpu_hist', predictor='gpu_predictor',)  ## Use GPU for training

xgb_model = xgb_regressor.fit(x_train_xg, 
                              y_train_log_xg, 
                              verbose=True, 
                              eval_set = [(x_train_xg, y_train_log_xg), (x_test_xg, y_test_log_xg)])
'''

end_time_xg_op = time.time()
training_time_xg_op = end_time_xg_op - start_time_xg_op

# Save the model
model_filename = 'xgb_optuna_model.json'
best_xgb.save_model(model_filename)
print(f'Model saved to: {model_filename}')

# Make predictions
y_pred_log_test_xg = best_xgb.predict(x_test_xg)
y_pred_log_train_xg = best_xgb.predict(x_train_xg)

y_pred_test_xg = np.exp(y_pred_log_test_xg)
y_pred_train_xg = np.exp(y_pred_log_train_xg)
y_train_xg = np.exp(y_train_log_xg)
y_test_xg = np.exp(y_test_log_xg)

r2_xgb = r2_score(y_test_xg, y_pred_test_xg)
mse_xgb = mean_squared_error(y_test_xg, y_pred_test_xg)
mae_xgb = mean_absolute_error(y_test_xg, y_pred_test_xg)
msle_xgb = mean_squared_log_error(y_test_xg, y_pred_test_xg)
mrse_xgb = mean_relative_squared_error(y_test_xg, y_pred_test_xg)

# Store and display metrics
xgb_eval_metrics = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'XGBoost_Op': [r2_xgb, mse_xgb, mae_xgb, msle_xgb, mrse_xgb, training_time_xg_op]
}

xgb_df_metrics = pd.DataFrame(xgb_eval_metrics)
xgb_df_metrics.to_csv('xgb_optuna_metrics.csv', index=False)

print(tabulate(xgb_df_metrics.round(4), headers='keys', tablefmt='pretty', showindex=False))

# Plot optimization history
optuna.visualization.plot_optimization_history(study_xgb)
plt.show()

[I 2025-08-25 09:21:01,382] A new study created in memory with name: no-name-c37ea34a-7b65-4dd0-bd39-86395bcb7785
[I 2025-08-25 09:21:01,816] Trial 0 finished with value: 0.2189086470434427 and parameters: {'max_depth': 96, 'learning_rate': 0.0040352687256245716, 'n_estimators': 133, 'min_child_weight': 2, 'gamma': 1.5221374711876635e-05, 'subsample': 0.4734784451609725, 'colsample_bytree': 0.5099043925731231, 'reg_alpha': 0.34740040686633034, 'reg_lambda': 0.10104629927023298}. Best is trial 0 with value: 0.2189086470434427.
[I 2025-08-25 09:21:04,653] Trial 1 finished with value: 0.0631651755627889 and parameters: {'max_depth': 65, 'learning_rate': 0.0043551161955877795, 'n_estimators': 816, 'min_child_weight': 4, 'gamma': 8.32806925132347e-07, 'subsample': 0.33768405942820434, 'colsample_bytree': 0.4547096942814426, 'reg_alpha': 1.2548050995663199e-06, 'reg_lambda': 4.214731749247084e-05}. Best is trial 1 with value: 0.0631651755627889.
[I 2025-08-25 09:21:05,682] Trial 2 finished w

Best XGBoost parameters: 
{'max_depth': 40, 'learning_rate': 0.013264923898431185, 'n_estimators': 427, 'min_child_weight': 4, 'gamma': 1.5298774820123473e-07, 'subsample': 0.5138456219205756, 'colsample_bytree': 0.7875371063648466, 'reg_alpha': 0.0016542034738830625, 'reg_lambda': 1.218455523681067e-05}
Best parameters saved to: xgb_optuna_best_params.json
[0]	validation_0-rmse:0.60646	validation_1-rmse:0.66004
[1]	validation_0-rmse:0.60052	validation_1-rmse:0.65466
[2]	validation_0-rmse:0.59452	validation_1-rmse:0.64858
[3]	validation_0-rmse:0.58871	validation_1-rmse:0.64332
[4]	validation_0-rmse:0.58293	validation_1-rmse:0.63911
[5]	validation_0-rmse:0.57629	validation_1-rmse:0.63292
[6]	validation_0-rmse:0.56988	validation_1-rmse:0.62817
[7]	validation_0-rmse:0.56329	validation_1-rmse:0.62194
[8]	validation_0-rmse:0.55681	validation_1-rmse:0.61613
[9]	validation_0-rmse:0.55117	validation_1-rmse:0.61180
[10]	validation_0-rmse:0.54526	validation_1-rmse:0.60733
[11]	validation_0-rmse:

/home/chris/venv/cdd/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning:

[09:22:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "eval_metrics" } are not used.




[26]	validation_0-rmse:0.46003	validation_1-rmse:0.54095
[27]	validation_0-rmse:0.45492	validation_1-rmse:0.53691
[28]	validation_0-rmse:0.45005	validation_1-rmse:0.53297
[29]	validation_0-rmse:0.44503	validation_1-rmse:0.52976
[30]	validation_0-rmse:0.43989	validation_1-rmse:0.52559
[31]	validation_0-rmse:0.43521	validation_1-rmse:0.52185
[32]	validation_0-rmse:0.43044	validation_1-rmse:0.51763
[33]	validation_0-rmse:0.42574	validation_1-rmse:0.51395
[34]	validation_0-rmse:0.42105	validation_1-rmse:0.50985
[35]	validation_0-rmse:0.41612	validation_1-rmse:0.50661
[36]	validation_0-rmse:0.41191	validation_1-rmse:0.50260
[37]	validation_0-rmse:0.40819	validation_1-rmse:0.49922
[38]	validation_0-rmse:0.40438	validation_1-rmse:0.49633
[39]	validation_0-rmse:0.40007	validation_1-rmse:0.49358
[40]	validation_0-rmse:0.39612	validation_1-rmse:0.49101
[41]	validation_0-rmse:0.39237	validation_1-rmse:0.48781
[42]	validation_0-rmse:0.38883	validation_1-rmse:0.48486
[43]	validation_0-rmse:0.38524	

In [ ]:
### TabPFN for Temperature Prediction

from tabpfn import TabPFNRegressor

start_time_tabpfn = time.time()

x_train_tab, x_test_tab, y_train_log_tab, y_test_log_tab = train_test_split(X, np.log(Y), test_size=0.20, random_state=42)

scaler = StandardScaler()
x_train_tab = scaler.fit_transform(x_train_tab)
x_test_tab = scaler.transform(x_test_tab)

tabpfn_regressor = TabPFNRegressor(
    device='cpu',  # Use 'cuda' if you have a GPU
)

tabpfn_regressor.fit(x_train_tab, y_train_log_tab)

y_pred_log_test = tabpfn_regressor.predict(x_test_tab)
y_pred_log_train = tabpfn_regressor.predict(x_train_tab)

y_pred_test = np.exp(y_pred_log_test)
y_pred_train = np.exp(y_pred_log_train)
y_train = np.exp(y_train_log_tab)
y_test = np.exp(y_test_log_tab)

end_time_tabpfn = time.time()
training_time_tabpfn = end_time_tabpfn - start_time_tabpfn

def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_tabpfn = r2_score(y_test, y_pred_test)
mse_tabpfn = mean_squared_error(y_test, y_pred_test)
mae_tabpfn = mean_absolute_error(y_test, y_pred_test)
msle_tabpfn = mean_squared_log_error(y_test, y_pred_test)
mrse_tabpfn = mean_relative_squared_error(y_test, y_pred_test)

tabpfn_eval_metrics = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'TabPFN Model': [r2_tabpfn, mse_tabpfn, mae_tabpfn, msle_tabpfn, mrse_tabpfn, training_time_tabpfn]
}

tabpfn_df_metrics = pd.DataFrame(tabpfn_eval_metrics)
tabpfn_df_metrics.to_csv('tabpfn_metrics.csv', index=False)

print(tabulate(tabpfn_df_metrics.round(4), headers='keys', tablefmt='pretty', showindex=False))


+---------------+--------------+
| Eval_metrics  | TabPFN Model |
+---------------+--------------+
|   R2 Score    |    0.8231    |
|      MSE      |  1334.5401   |
|      MAE      |    17.991    |
|     MSLE      |    0.1016    |
|     MRSE      |    0.3247    |
| Training time |    7.4432    |
+---------------+--------------+
